In [1]:
!pip install hyperopt

In [0]:
import os
import datetime

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

import matplotlib.pyplot as plt

from skimage import color, exposure

In [0]:
%load_ext tensorboard
%matplotlib inline

In [4]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix_road_sign/dw_matrix_road_sign"

/content/drive/My Drive/Colab Notebooks/dw_matrix_road_sign/dw_matrix_road_sign


In [0]:
signs = pd.read_csv('data/signnames.csv')
labels_dict = signs.to_dict()['b']

train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  log_dir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%s'))
  tensordboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensordboard_callback]
      )
  
  return model

def predict(trained_model, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  
  y_pred_prob = trained_model.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

def train_and_predict(model, X_train, y_train, X_test, y_test):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')
  ])

In [9]:
train_and_predict(get_cnn_v5(input_shape, num_classes), X_train, y_train, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.7528 - accuracy: 0.2556 - val_loss: 0.9873 - val_accuracy: 0.6955
Epoch 2/5
272/272 [==============================] - 4s 13ms/step - loss: 0.8275 - accuracy: 0.7354 - val_loss: 0.1820 - val_accuracy: 0.9496
Epoch 3/5
272/272 [==============================] - 4s 13ms/step - loss: 0.3514 - accuracy: 0.8929 - val_loss: 0.0799 - val_accuracy: 0.9763
Epoch 4/5
272/272 [==============================] - 4s 13ms/step - loss: 0.2189 - accuracy: 0.9333 - val_loss: 0.0537 - val_accuracy: 0.9838
Epoch 5/5
272/272 [==============================] - 4s 13ms/step - loss: 0.1783 - accuracy: 0.9485 - val_loss: 0.0290 - val_accuracy: 0.9918


0.9816326530612245

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      )
  
  accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [24]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7)
}

best = fmin(func_obj, space, tpe.suggest, 25, Trials())

{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3063333438647573, 'dropout_cnn_block_three': 0.430711624467245, 'dropout_cnn_block_two': 0.4582359375067563, 'dropout_dense_block_one': 0.4611466915013519, 'dropout_dense_block_two': 0.45846252225928225}
accuracy=0.9517006874084473
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4017271378876417, 'dropout_cnn_block_three': 0.4908162211159742, 'dropout_cnn_block_two': 0.4707529599081126, 'dropout_dense_block_one': 0.4223238663875107, 'dropout_dense_block_two': 0.6171748861562942}
accuracy=0.9274376630783081
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.31347447827709884, 'dropout_cnn_block_three': 0.32269384846141247, 'dropout_cnn_block_two': 0.4311975011821987, 'dropout_dense_block_one': 0.44709490219986175, 'dropout_dense_block_two': 0.6248536364993673}
accuracy=0.962358295917511
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.4846519771562247, 'dropout_cnn_block_three': 0.3059637787232156, 'dropout_cnn_block_two': 0.40375967245246